In [ ]:
import numpy as np

class MatrixFactorization:
    def __init__(self, R, K=2, alpha=0.001, beta=0.02, iterations=100):
        """
        Initialize Matrix Factorization model
        
        Parameters:
        R: rating matrix (n_users × n_items)
        K: number of latent features
        alpha: learning rate
        beta: regularization parameter
        iterations: number of iterations for gradient descent
        """
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        
    def train(self):
        """Train the model using gradient descent"""
        # Initialize user and item latent feature matrices randomly
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Create a mask for non-zero elements
        self.mask = (self.R > 0).astype(float)
        
        # Training loop
        for iteration in range(self.iterations):
            # Compute current predictions
            prediction = self.predict()
            
            # Compute error only for observed ratings
            error = self.mask * (self.R - prediction)
            
            # Calculate total error to track convergence
            total_error = np.sum(error ** 2) + \
                         self.beta * (np.sum(self.P ** 2) + np.sum(self.Q ** 2))
            
            if iteration % 10 == 0:
                print(f"Iteration: {iteration}, Error: {total_error}")
            
            # Update P and Q matrices using gradient descent
            for u in range(self.num_users):
                for i in range(self.num_items):
                    if self.mask[u, i] > 0:
                        # Calculate error for this rating
                        e_ui = error[u, i]
                        
                        # Update user and item latent features
                        self.P[u, :] += self.alpha * (2 * e_ui * self.Q[i, :] - self.beta * self.P[u, :])
                        self.Q[i, :] += self.alpha * (2 * e_ui * self.P[u, :] - self.beta * self.Q[i, :])
    
    def predict(self, u=None, i=None):
        """
        Make predictions for either a specific user-item pair or all pairs
        """
        if u is not None and i is not None:
            # Predict single rating
            return self.P[u, :].dot(self.Q[i, :].T)
        else:
            # Predict all ratings
            return self.P.dot(self.Q.T)
    
    def get_recommendations(self, user_id, n_recommendations=5):
        """
        Get top N recommendations for a specific user
        """
        # Get all predictions for this user
        user_pred = self.predict(user_id)
        
        # Create a mask for items the user hasn't rated
        unrated_items = self.R[user_id] == 0
        
        # Get predictions for unrated items
        pred_unrated = user_pred[unrated_items]
        items_unrated = np.where(unrated_items)[0]
        
        # Sort predictions and get top N
        top_items_idx = pred_unrated.argsort()[-n_recommendations:][::-1]
        recommendations = items_unrated[top_items_idx]
        
        return [(item, user_pred[item]) for item in recommendations]

# Example usage
def main():
    # Create a sample rating matrix (users × items)
    R = np.array([
        [5, 3, 0, 1],
        [4, 0, 0, 1],
        [1, 1, 0, 5],
        [1, 0, 0, 4],
        [0, 1, 5, 4],
    ])
    
    # Initialize and train the model
    mf = MatrixFactorization(R, K=2, alpha=0.001, beta=0.02, iterations=100)
    mf.train()
    
    # Get recommendations for user 0
    recommendations = mf.get_recommendations(0, n_recommendations=2)
    print("\nTop 2 recommendations for user 0:")
    for item, pred_rating in recommendations:
        print(f"Item {item}: Predicted rating = {pred_rating:.2f}")
    
    # Make a specific prediction
    user_id, item_id = 0, 2
    prediction = mf.predict(user_id, item_id)
    print(f"\nPredicted rating for user {user_id}, item {item_id}: {prediction:.2f}")

if __name__ == "__main__":
    main()